# Tests with `cadquery`

In [ ]:
import cadquery as cq

In [ ]:
step_file = 'Stage5_704MHz_cavity.stp'
result = cq.importers.importStep("009_muCol_cavity_beta/cst/" + step_file)

In [ ]:
display(result)

In [ ]:
import numpy as np
import re

def extract_solid_names_and_materials(stp_file):
    # Load file lines as an array of strings
    lines = np.genfromtxt(stp_file, dtype=str, delimiter="\n", encoding="utf-8", comments=None)

    solid_dict = {}
    material_dict = {}

    # Regex patterns
    solid_pattern = re.compile(r"#(\d+)=MANIFOLD_SOLID_BREP\('([^']+)'.*;")
    material_pattern = re.compile(r"#(\d+)=PRESENTATION_LAYER_ASSIGNMENT\('([^']+)','[^']+',\(#([\d#,]+)\)\);")

    # Extract solids
    for line in lines:
        solid_match = solid_pattern.search(line)
        if solid_match:
            solid_number = int(solid_match.group(1))
            solid_name = solid_match.group(2)
            solid_dict[solid_number] = solid_name

    # Extract materials
    for line in lines:
        material_match = material_pattern.search(line)
        if material_match:
            material_name = material_match.group(2)
            solid_numbers = [int(num.strip("#")) for num in material_match.group(3).split(',')] # Extract numbers as a list
            
            for solid_number in solid_numbers:
                if solid_number in solid_dict:  # Only assign if it's a known solid
                    material_dict[solid_number] = material_name

    return solid_dict, material_dict

# Example usage
stp_file = "009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp"
solids, materials = extract_solid_names_and_materials(stp_file)

# Print results
print("Solids:", solids)
print("Materials:", materials)


In [ ]:
# Save each object in STL file
stp_file = "009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp"
solids, materials = extract_solid_names_and_materials(stp_file)

for i, obj in enumerate(result.objects[0]):
    solid = solids[list(solids.keys())[i]].strip(' ')
    mat = materials[list(solids.keys())[i]].strip(' ')

    solid = re.sub(r'[^a-zA-Z0-9_-]', '-', solid)
    mat = re.sub(r'[^a-zA-Z0-9_-]', '-', mat)

    print(f'{str(i).zfill(3)}_{solid}_{mat}'+".stl")
    obj.exportStl('009_muCol_cavity_beta/cst/'+f'{str(i).zfill(3)}_{solid}_{mat}'+".stl") 
    #obj.exportStl('009_muCol_cavity_beta/cst/'+str(i)+".stl") 
    #display(obj)

In [ ]:
# Create a sample assembly
assy = cq.Assembly()
body = cq.Workplane().box(10, 10, 10)
assy.add(body, color=cq.Color(1, 0, 0), name="body")
pin = cq.Workplane().center(2, 2).cylinder(radius=2, height=20)
assy.add(pin, color=cq.Color(0, 1, 0), name="pin")

In [ ]:
# Save the assembly to STEP
assy.export("out.stp", "STEP", mode="fused", glue=True, write_pcurves=False)

In [ ]:
out = cq.importers.importStep('out.stp')

In [ ]:
display(out)

# Tests with `trimesh`

In [ ]:
import cascadio
import trimesh

In [ ]:
cascadio.step_to_obj(
            'out.stp',
            'out.obj',
            use_colors=True,
        )

In [ ]:
mesh = trimesh.load('out.obj', merge_primitives=True)

In [ ]:
mesh.show()

In [ ]:
import pyvista as pv
out = pv.read('out.obj')

In [ ]:
out.plot()

In [ ]:
pl = pv.Plotter()
pl.add_mesh(out, scalars='mat_1')
pl.show()

# Final tests with wakis funcs

In [1]:
import sys 
sys.path.append('../wakis')

from wakis import geometry

In [4]:
stl_solids = geometry.generate_stl_solids_from_stp('009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp')

Generating stl from file: 009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp... 
000_Vacuum-Half_cell_dx_Vacuum.stl.stl
001_Be-windows-Be-window-left_Berillium.stl.stl
002_Be-windows-Be-window-right_Berillium.stl.stl
003_Walls-Cavity-walls_Copper--annealed-.stl.stl
004_Vacuum-Half_cell_sx_Vacuum.stl.stl


In [3]:
import numpy as np
import re

def extract_solid_colors(stp_file):
    """
    Extracts the color (RGB values) assigned to each solid in a STEP (STP) file.

    Args:
        stp_file (str): Path to the STP file.

    Returns:
        dict: A dictionary where keys are solid names, and values are (R, G, B) tuples.
    """

    # Load file lines as an array of strings
    lines = np.genfromtxt(stp_file, dtype=str, delimiter="\n", encoding="utf-8", comments=None)

    # Dictionaries to store parsed data
    solid_dict = {}  # {solid_id: solid_name}
    color_dict = {}  # {color_id: (R, G, B)}
    solid_color_map = {}  # {solid_name: (R, G, B)}

    # Regular expressions for each component
    solid_pattern = re.compile(r"#(\d+)=MANIFOLD_SOLID_BREP\('([^']+)'.*;")
    style_pattern = re.compile(r"#(\d+)=STYLED_ITEM\('?',\(#(\d+)\),#(\d+)\);")
    assignment_pattern = re.compile(r"#(\d+)=PRESENTATION_STYLE_ASSIGNMENT\(\(#(\d+)\)\);")
    rendering_pattern = re.compile(r"#(\d+)=SURFACE_STYLE_RENDERING\(#(\d+),.*\);")
    color_pattern = re.compile(r"#(\d+)=COLOUR_RGB\('?',([\d.]+),([\d.]+),([\d.]+)\);")

    # Step 1: Extract Solids
    for line in lines:
        solid_match = solid_pattern.search(line)
        if solid_match:
            solid_id = int(solid_match.group(1))
            solid_name = solid_match.group(2)
            solid_dict[solid_id] = solid_name

    # Step 2: Extract Color References from STYLED_ITEM
    styled_map = {}  # {solid_id: style_id}
    for line in lines:
        style_match = style_pattern.search(line)
        if style_match:
            style_id = int(style_match.group(2))
            solid_id = int(style_match.group(3))
            styled_map[solid_id] = style_id

    # Step 3: Extract Style Assignments
    style_map = {}  # {style_id: rendering_id}
    for line in lines:
        assign_match = assignment_pattern.search(line)
        if assign_match:
            rendering_id = int(assign_match.group(2))
            style_id = int(assign_match.group(1))
            style_map[style_id] = rendering_id

    # Step 4: Extract Rendering Assignments
    rendering_map = {}  # {rendering_id: color_id}
    for line in lines:
        render_match = rendering_pattern.search(line)
        if render_match:
            rendering_id = int(render_match.group(1))
            color_id = int(render_match.group(2))
            rendering_map[rendering_id] = color_id

    # Step 5: Extract Colors
    for line in lines:
        color_match = color_pattern.search(line)
        if color_match:
            color_id = int(color_match.group(1))
            r, g, b = float(color_match.group(2)), float(color_match.group(3)), float(color_match.group(4))
            color_dict[color_id] = (r, g, b)

    # Step 6: Map Solids to Colors
    for solid_id, solid_name in solid_dict.items():
        if solid_id in styled_map:
            style_id = styled_map[solid_id]
            if style_id in style_map:
                rendering_id = style_map[style_id]
                if rendering_id in rendering_map:
                    color_id = rendering_map[rendering_id]
                    if color_id in color_dict:
                        solid_color_map[solid_name] = color_dict[color_id]

    return solid_color_map

# Example usage
stp_file = "009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp"
solid_colors = extract_solid_colors(stp_file)

# Print results
for solid, color in solid_colors.items():
    print(f"Solid: {solid}, Color: {color}")

In [6]:
solid_colors

{}

In [ ]:
import re
import numpy as np

def extract_solid_colors(stp_file):
    """
    Extracts the color (RGB values) assigned to each solid in a STEP (STP) file.

    Args:
        stp_file (str): Path to the STP file.

    Returns:
        dict: A dictionary where keys are solid names, and values are (R, G, B) tuples.
    """

    # Load file lines as an array of strings
    lines = np.genfromtxt(stp_file, dtype=str, delimiter="\n", encoding="utf-8", comments=None)

    # Dictionaries to store parsed data
    solid_dict = {}  # {solid_id: solid_name}
    color_dict = {}  # {color_id: (R, G, B)}
    solid_color_map = {}  # {solid_name: (R, G, B)}

    # Regular expressions for each component
    solid_pattern = re.compile(r"#(\d+)=MANIFOLD_SOLID_BREP\('([^']+)'.*;")
    style_pattern = re.compile(r"#(\d+)=STYLED_ITEM\('?',\(#(\d+)\),#(\d+)\);")
    assignment_pattern = re.compile(r"#(\d+)=PRESENTATION_STYLE_ASSIGNMENT\(\(#(\d+)\)\);")
    rendering_pattern = re.compile(r"#(\d+)=SURFACE_STYLE_RENDERING\(#(\d+),.*\);")
    color_pattern = re.compile(r"#(\d+)=COLOUR_RGB\('?',([\d.]+),([\d.]+),([\d.]+)\);")

    # Step 1: Extract Solids
    for line in lines:
        solid_match = solid_pattern.search(line)
        if solid_match:
            solid_id = int(solid_match.group(1))
            solid_name = solid_match.group(2)
            solid_dict[solid_id] = solid_name

    print(f"Extracted {len(solid_dict)} solids: {solid_dict}")  # DEBUGGING

    # Step 2: Extract Style Assignments
    styled_map = {}  # {solid_id: style_id}
    for line in lines:
        style_match = style_pattern.search(line)
        if style_match:
            style_id = int(style_match.group(2))
            solid_id = int(style_match.group(3))
            styled_map[solid_id] = style_id

    print(f"Extracted {len(styled_map)} style assignments: {styled_map}")  # DEBUGGING

    # Step 3: Extract Style-Rendering Mapping
    style_map = {}  # {style_id: rendering_id}
    for line in lines:
        assign_match = assignment_pattern.search(line)
        if assign_match:
            rendering_id = int(assign_match.group(2))
            style_id = int(assign_match.group(1))
            style_map[style_id] = rendering_id

    print(f"Extracted {len(style_map)} style-to-rendering mappings: {style_map}")  # DEBUGGING

    # Step 4: Extract Rendering-Color Mapping
    rendering_map = {}  # {rendering_id: color_id}
    for line in lines:
        render_match = rendering_pattern.search(line)
        if render_match:
            rendering_id = int(render_match.group(1))
            color_id = int(render_match.group(2))
            rendering_map[rendering_id] = color_id

    print(f"Extracted {len(rendering_map)} rendering-to-color mappings: {rendering_map}")  # DEBUGGING

    # Step 5: Extract Colors
    for line in lines:
        color_match = color_pattern.search(line)
        if color_match:
            color_id = int(color_match.group(1))
            r, g, b = float(color_match.group(2)), float(color_match.group(3)), float(color_match.group(4))
            color_dict[color_id] = (r, g, b)

    print(f"Extracted {len(color_dict)} colors: {color_dict}")  # DEBUGGING

    # Step 6: Map Solids to Colors
    for solid_id, solid_name in solid_dict.items():
        if solid_id in styled_map:
            style_id = styled_map[solid_id]
            if style_id in style_map:
                rendering_id = style_map[style_id]
                if rendering_id in rendering_map:
                    color_id = rendering_map[rendering_id]
                    if color_id in color_dict:
                        solid_color_map[solid_name] = color_dict[color_id]

    print(f"Final solid-to-color mapping: {solid_color_map}")  # DEBUGGING

    return solid_color_map

# Example usage
stp_file = "009_muCol_cavity_beta/cst/Stage5_704MHz_cavity.stp"
solid_colors = extract_solid_colors(stp_file)
